## WEB SCRAPING FOR THE E-14 FORMATS

In [1]:
if False:
    !pip install beautifulsoup
    !pip install selenium

  Using cached https://files.pythonhosted.org/packages/40/f2/6c9f2f3e696ee6a1fb0e4d7850617e224ed2b0b1e872110abffeca2a09d4/BeautifulSoup-3.2.2.tar.gz


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\gd.orbegozo10\AppData\Local\Continuum\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\GDA7A5~1.ORB\\AppData\\Local\\Temp\\pip-install-kwsi9km9\\beautifulsoup\\setup.py'"'"'; __file__='"'"'C:\\Users\\GDA7A5~1.ORB\\AppData\\Local\\Temp\\pip-install-kwsi9km9\\beautifulsoup\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base pip-egg-info
         cwd: C:\Users\GDA7A5~1.ORB\AppData\Local\Temp\pip-install-kwsi9km9\beautifulsoup\
    Complete output (6 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\GDA7A5~1.ORB\AppData\Local\Temp\pip-install-kwsi9km9\beautifulsoup\setup.py", line 3
        "You're trying to run a very old release of Beautiful Soup under Python 3. This will 

In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
import urllib
import pandas as pd
import time
import re
import matplotlib.pyplot as plt
import numpy as np
import sys
import random
import os

2018 presidential elections: https://elecciones1.registraduria.gov.co/e14_pre1_2018/

## Functions

In [3]:
def main(html, dept_list, data_frame_dir='formularios_e_14/'):
  
    soup = BeautifulSoup(html)
    
    dept_select = Select(browser.find_element_by_id('select_dep'))
    
    pdf_info = pd.DataFrame(columns=['dept', 'mun', 'zona', 'pto','n_format','format_names'])
    
    # Choose department
    for i_d,dept in enumerate(dept_list):
       
        mun_select, mun_values = find_select(dept_select, dept, 0)
        
        # Create a directory for each department
        dept_dir = fr'formularios_e_14/{dept}'
        if not os.path.exists(dept_dir):
            os.mkdir(dept_dir)
        
        # Choose municipality
        for i_m,mun in enumerate(mun_values):

            zona_select, zona_values = find_select(mun_select, mun, 1)
               
            # Create a directory for each municipality
            mun_dir = fr'formularios_e_14/{dept}/{mun}'
            if not os.path.exists(mun_dir):
                os.mkdir(mun_dir)
            print(f'\nDept: {dept} - Mun: {mun} ({(i_m+1)}/{len(mun_values)})')
            
            # Choose zona
            for i_z,zona in enumerate(zona_values):
                
                pto_select, pto_values = find_select(zona_select, zona, 2)
                
                # Create a directory for each zona
                zona_dir = fr'formularios_e_14/{dept}/{mun}/{zona}'
                if not os.path.exists(zona_dir):
                    os.mkdir(zona_dir)
                                
                progressBar((i_z+1), len(zona_values))
                
                # Choose puesto and get pdf links from each voting table
                for i_p, pto in enumerate(pto_values):
                    
                    pdf_list = find_select(pto_select, pto, 3)

                    # Download the pdfs
                    for i,link in enumerate(pdf_list):
                        time.sleep(random.normalvariate(2, 0.4))
                        file_name = link.split('/')[-1]

                        pdf_name = f'{zona_dir}/{file_name}'

                        urllib.request.urlretrieve(link,pdf_name)

                    # Put information inside the dataframe
                    n_format = len(pdf_list)
                    format_names = ','.join([x.split('/')[-1].split('_')[0] for i,x in enumerate(pdf_list)])
                    pdf_info = pdf_info.append({'dept': dept, 'mun': mun, 
                                                'zona': zona , 'pto': pto, 
                                                'n_format': n_format, 
                                                'format_names': format_names},
                                               ignore_index=True)

            # Save the dataframe after each municipality
            pdf_info.to_csv(f'{data_frame_dir}info_formatos_{dept}.csv', index=False, sep=';')
    
    return pdf_info

In [4]:
def find_select(select, value, index, sleep_time_mu=2, sleep_time_de=0.4):
    
    # Names for the levels of information inside the html
    id_list = ['mpio', 'zona', 'pto']
    
    # Find the given value inside the given select
    select.select_by_value(value)
    
    # Time to wait between searchas
    sleep_time = random.normalvariate(sleep_time_mu, sleep_time_de)
    time.sleep(sleep_time)

    # Read html for the current level
    html = browser.page_source
    soup = BeautifulSoup(html)
    
    # index == 3 is the place to stop and look for the pdf_links
    if index < 3:
        option_list = str(soup.find_all('div',{'class': 'form-group'})[index])
        values = re.findall('<option.+?="([0-9]+)"', option_list)

        select = Select(browser.find_element_by_id(id_list[index]))
    
        return select, values
    
    elif index == 3:
        mesa_list = soup.find_all('a', {'href':re.compile('https.*pdf')})
        pdf_links = re.findall('href="(https.+?pdf)"', str(mesa_list))
             
        return pdf_links

In [5]:
def progressBar(value, endvalue, bar_length=40):
    percent = float(value) / endvalue
    arrow = '-' * int(round(percent * bar_length)-1) + '>'
    spaces = ' ' * (bar_length - len(arrow))

    sys.stdout.write(f"\rProgreso: [{arrow + spaces}] {int(round(percent * 100))}% ({value} de {endvalue} zonas.)")
    sys.stdout.flush()

### RUN MAIN

In [6]:
dept_values = ['1', '3', '5', '7', '9', '11', '12', '13', '15', '16', '17', '19', '21',
               '23', '24', '25', '26', '27', '28', '29', '31', '40', '44', '46', '48',
               '50', '52', '54', '56', '60', '64', '68', '72', '88']

## CAMBIAR

In [13]:
#os.chdir(r'D:\Dropbox\Proyectos\python\formatos_elecciones')
os.chdir(r'C:\Users\gd.orbegozo10\Dropbox\Cosas_Programacion\Python\python\formatos_elecciones')
service = Service('C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe')

## CORRER

In [20]:
pdf_dir = 'formularios_e_14/pdf_no_text/'
png_dir = 'formularios_e_14/png_no_text/'

In [21]:
pdf_info = pd.DataFrame(columns=['dept', 'mun', 'zona', 'pto','n_format','format_names'])

service.start()
browser = webdriver.Remote(service.service_url)

url = 'https://elecciones1.registraduria.gov.co/e14_pre1_2018/'

browser.maximize_window()

browser.get(url)

html = browser.page_source

pdf_info = main(html, ['60', '64', '68', '72', '88'])


Dept: 60 - Mun: 1 (1/11)
Progreso: [--------------------------------------->] 100% (4 de 4 zonas.)
Dept: 60 - Mun: 7 (2/11)
Progreso: [--------------------------------------->] 100% (2 de 2 zonas.)
Dept: 60 - Mun: 10 (3/11)
Progreso: [--------------------------------------->] 100% (1 de 1 zonas.)
Dept: 60 - Mun: 13 (4/11)
Progreso: [--------------------------------------->] 100% (1 de 1 zonas.)
Dept: 60 - Mun: 16 (5/11)
Progreso: [--------------------------------------->] 100% (1 de 1 zonas.)
Dept: 60 - Mun: 17 (6/11)
Progreso: [--------------------------------------->] 100% (1 de 1 zonas.)
Dept: 60 - Mun: 19 (7/11)
Progreso: [--------------------------------------->] 100% (2 de 2 zonas.)
Dept: 60 - Mun: 21 (8/11)
Progreso: [--------------------------------------->] 100% (1 de 1 zonas.)
Dept: 60 - Mun: 22 (9/11)
Progreso: [--------------------------------------->] 100% (1 de 1 zonas.)
Dept: 60 - Mun: 30 (10/11)
Progreso: [--------------------------------------->] 100% (1 de 1 zonas.)


WebDriverException: Message: chrome not reachable
  (Session info: chrome=84.0.4147.135)


### DEBUG

### How to find a single link (the loops and functions generalize this) 

In [9]:
sleep_time = 1

In [24]:
service = Service('C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe')
service.start()
browser = webdriver.Remote(service.service_url)

url = 'https://elecciones1.registraduria.gov.co/e14_pre1_2018/'

browser.maximize_window()

browser.get(url)

html = browser.page_source
soup = BeautifulSoup(html)

dept_select = Select(browser.find_element_by_id('select_dep'))

In [25]:
dept = '1'
dept_select.select_by_value(dept)

dept_html = browser.page_source
dept_soup = BeautifulSoup(dept_html)

time.sleep(sleep_time)

dept_html = browser.page_source
dept_soup = BeautifulSoup(dept_html)

mun_option_list = str(dept_soup.find_all('div',{'class': 'form-group'})[0])
mun_values = re.findall('<option.+?="([0-9]+)">', mun_option_list)

mun_select = Select(browser.find_element_by_id('mpio'))

mun = '250'
mun_select.select_by_value(mun) 

time.sleep(sleep_time)

mun_html = browser.page_source
mun_soup = BeautifulSoup(mun_html)

zona_option_list = str(mun_soup.find_all('div',{'class': 'form-group'})[1])
zona_values = re.findall('<option.+?="([0-9]+)"', zona_option_list)

zona_select = Select(browser.find_element_by_id('zona'))

zona = '01'
zona_select.select_by_value(zona)

time.sleep(sleep_time)

zona_html = browser.page_source
zona_soup = BeautifulSoup(zona_html)

pto_option_list = str(zona_soup.find_all('div',{'class': 'form-group'})[2])
pto_values = re.findall('<option.+?="([0-9]+)">', pto_option_list)

pto_select = Select(browser.find_element_by_id('pto'))

pto = pto_values[0]
pto_select.select_by_value(pto)

time.sleep(sleep_time)

In [26]:
zona_values

['01', '02', '98', '99']

In [28]:
pto_values

['01', '02']

In [17]:
print('A')
print('\nA\rCV')

A

CV


In [480]:
mesa_html = browser.page_source
mesa_soup = BeautifulSoup(mesa_html)

mesa_list = mesa_soup.find_all('a', {'href':re.compile('https.*pdf')})

pdf_links = re.findall('href="(https.+?pdf)"', str(mesa_list))

In [482]:
link = pdf_links[10]
link

'https://elecciones1.registraduria.gov.co/e14_pre1_2018//e14_divulgacion/28/100/000/PRE/5903613_E14_PRE_X_28_100_000_XX_00_011_X_XXX.pdf'

### Scraping inside a single loop

In [3]:
sleep_time = abs(random.normalvariate(2,0.3))

service = Service('C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe')
service.start()
browser = webdriver.Remote(service.service_url)

url = 'https://elecciones1.registraduria.gov.co/e14_pre1_2018/'

browser.maximize_window()

browser.get(url)

html = browser.page_source
soup = BeautifulSoup(html)

option_list = soup.find_all('option',{'value': re.compile('[0-9]+')})
option_value = re.findall('"([0-9]+)"',str(option_list))

first_value = [i for i,x in enumerate(option_value) if (x == '1') | (x == '01')]

dept_list = option_list[first_value[0]:first_value[1]]
dept_values = re.findall('<option value="(\d+)"', str(dept_list))

dept_select = Select(browser.find_element_by_id('select_dep'))

for i_d,dept in enumerate(dept_values):
    dept_select.select_by_value(dept)
    
    time.sleep(sleep_time)

    dept_html = browser.page_source
    dept_soup = BeautifulSoup(dept_html)

    mun_option_list = str(dept_soup.find_all('div',{'class': 'form-group'})[0])
    mun_values = re.findall('<option.+="([0-9]+)">', mun_option_list)

    mun_select = Select(browser.find_element_by_id('mpio'))
    
    for i_m,mun in enumerate(mun_values):
        mun_select.select_by_value(mun) 
        
        time.sleep(sleep_time)

        mun_html = browser.page_source
        mun_soup = BeautifulSoup(mun_html)
        
        zona_option_list = str(mun_soup.find_all('div',{'class': 'form-group'})[1])
        zona_values = re.findall('<option value="([0-9]+)">', zona_option_list)
        
        zona_select = Select(browser.find_element_by_id('zona'))
        
        for i_z,zona in enumerate(zona_values):
            zona_select.select_by_value(zona)
            
            time.sleep(sleep_time)
            
            zona_html = browser.page_source
            zona_soup = BeautifulSoup(zona_html)

            pto_option_list = str(zona_soup.find_all('div',{'class': 'form-group'})[2])
            pto_values = re.findall('<option value="([0-9]+)">', pto_option_list)
            
            pto_select = Select(browser.find_element_by_id('pto'))
            
            for i_p,pto in enumerate(pto_values):
                pto_select.select_by_value(pto)
                
                time.sleep(sleep_time)
                
                mesa_html = browser.page_source
                mesa_soup = BeautifulSoup(mesa_html)

                mesa_list = mesa_soup.find_all('a', {'href':re.compile('https.*pdf')})

                pdf_links = re.findall('href="(https.+?pdf)"', str(mesa_list))
                
                for i_l,link in enumerate(pdf_links):
                    file_name = link.split('/')[-1]
                    
                    pdf_name = f'{pdf_dir}{file_name}'
                    
                    #urllib.request.urlretrieve(link,pdf_name)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=84.0.4147.105)


In [29]:
dept_list = ['1']

pdf_info = pd.DataFrame(columns=['dept', 'mun', 'zona', 'pto','n_format','format_names'])

service = Service('C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe')
service.start()
browser = webdriver.Remote(service.service_url)

url = 'https://elecciones1.registraduria.gov.co/e14_pre1_2018/'

browser.maximize_window()

browser.get(url)

html = browser.page_source
soup = BeautifulSoup(html)

dept_select = Select(browser.find_element_by_id('select_dep'))

# Choose department
for i_d,dept in enumerate(dept_list):

    mun_select, mun_values = find_select(dept_select, dept, 0)

    # Choose municipality
    for i_d,mun in enumerate(mun_values):

        zona_select, zona_values = find_select(mun_select, mun, 1)

        # Choose zona
        for i_z,zona in enumerate(zona_values):
            pto_select, pto_values = find_select(zona_select, zona, 2)

            # Choose puesto and get pdf links from each voting table
            pto = pto_values[0]
            pdf_list = find_select(pto_select, pto, 3)

            # Download the pdfs
            for i,link in enumerate(pdf_list):
                file_name = link.split('/')[-1]

                pdf_name = f'{pdf_dir}{file_name}'

                urllib.request.urlretrieve(link,pdf_name)

            # Put information inside the dataframe
            n_format = len(pdf_list)
            format_names = ';'.join([x.split('/')[-1].split('_')[0] for i,x in enumerate(pdf_list)])
            pdf_info = pdf_info.append({'dept': dept, 'mun': mun, 
                                        'zona': zona , 'pto': pto, 
                                        'n_format': n_format, 
                                        'format_names': format_names},
                                       ignore_index=True)

        # Save the dataframe each municipality
        pdf_info.to_csv(f'{data_frame_dir}','info_formatos.csv')


NoSuchElementException: Message: Cannot locate option with value: 1


In [12]:
service = Service('C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe')
service.start()
browser = webdriver.Remote(service.service_url)

url = 'https://elecciones1.registraduria.gov.co/e14_pre1_2018/'

browser.maximize_window()

browser.get(url)

html = browser.page_source
soup = BeautifulSoup(html)

pdf_info = pd.DataFrame(columns=['dept', 'mun', 'zona', 'pto','n_format','format_names'])


dept_select = Select(browser.find_element_by_id('select_dep'))

# Choose department
dept = '1'

mun_select, mun_values = find_select(dept_select, dept, 0)

# Choose municipality
mun = mun_values[20]

zona_select, zona_values = find_select(mun_select, mun, 1)

# Choose zona
zona = zona_values[0] 

pto_select, pto_values = find_select(zona_select, zona, 2)

# Choose puesto and get pdf links from each voting table
pto = pto_values[0]
pdf_list = find_select(pto_select, pto, 3)

# Download the pdfs
for i,link in enumerate(pdf_list):
    file_name = link.split('/')[-1]

    pdf_name = f'{pdf_dir}{file_name}'

    urllib.request.urlretrieve(link,pdf_name)
    
n_format = len(pdf_list)
format_names = '-'.join([x.split('/')[-1].split('_')[0] for i,x in enumerate(pdf_list)])
pdf_info = pdf_info.append({'dept': dept, 'mun': mun, 
                            'zona': zona , 'pto': pto, 
                            'n_format': n_format, 
                            'format_names': format_names},
                           ignore_index=True)

In [13]:
pdf_list

['https://elecciones1.registraduria.gov.co/e14_pre1_2018//e14_divulgacion/01/055/000/PRE/8280914_E14_PRE_X_01_055_000_XX_00_001_X_XXX.pdf',
 'https://elecciones1.registraduria.gov.co/e14_pre1_2018//e14_divulgacion/01/055/000/PRE/2092115_E14_PRE_X_01_055_000_XX_00_002_X_XXX.pdf',
 'https://elecciones1.registraduria.gov.co/e14_pre1_2018//e14_divulgacion/01/055/000/PRE/4818516_E14_PRE_X_01_055_000_XX_00_003_X_XXX.pdf',
 'https://elecciones1.registraduria.gov.co/e14_pre1_2018//e14_divulgacion/01/055/000/PRE/6213411_E14_PRE_X_01_055_000_XX_00_004_X_XXX.pdf',
 'https://elecciones1.registraduria.gov.co/e14_pre1_2018//e14_divulgacion/01/055/000/PRE/6048718_E14_PRE_X_01_055_000_XX_00_005_X_XXX.pdf',
 'https://elecciones1.registraduria.gov.co/e14_pre1_2018//e14_divulgacion/01/055/000/PRE/8875015_E14_PRE_X_01_055_000_XX_00_006_X_XXX.pdf',
 'https://elecciones1.registraduria.gov.co/e14_pre1_2018//e14_divulgacion/01/055/000/PRE/3222116_E14_PRE_X_01_055_000_XX_00_007_X_XXX.pdf',
 'https://elecciones